In [1]:
function number_to_binary(number::Int; bits::Int=9)::String
    out = collect('.'^bits)
    remaining = number
    while remaining > 0
        l2 = Int(floor(log2(remaining)))
        replace
        out[end-l2] = '#'
        remaining -= 2^l2
    end
    return join(out)
end

function expand_image(image::Matrix{Char}, char::Char)::Matrix{Char}
    nrow = permutedims(collect(char^size(image)[2]))
    image = vcat(nrow, nrow, image, nrow, nrow)
    ncol = collect(char^size(image)[1])
    image = hcat(ncol, ncol, image, ncol, ncol)
    return image
end

function solve(filename, npasses)

    lines = readlines(filename)
    image = Matrix{Char}(permutedims(hcat(collect.(lines[3:end])...)))

    algorithm = lines[1]

    algorithm_map = Dict{String,Char}([])
    for (i, c) in enumerate(algorithm)
        n = number_to_binary(i-1)
        algorithm_map[n] = c
    end

    passes_done = 0
    expand_with = '.'

    while passes_done < npasses
        if passes_done != 0
            # Expand with whatever empty space would be now
            expand_with = algorithm_map[expand_with^9]
        end
        image = Matrix{Char}(expand_image(image, expand_with))
        nimage = Matrix{Char}(copy(image))
        # This loop is slower than I would have 
        # guessed without -O3
        for i=2:size(image)[1]-1
            for j=2:size(image)[2]-1
                input_pixels = Vector{Char}([])
                for k=(i-1):(i+1)
                    for l=(j-1):(j+1)
                        push!(input_pixels, image[k, l])
                    end
                end
                nimage[i, j] = algorithm_map[String(input_pixels)]
            end
        end
        image = nimage[2:end-1, 2:end-1]
        passes_done += 1
    end

   return count(x->x=='#', vcat(image))
end

solve (generic function with 1 method)

# Part 1

In [2]:
println("Part 1 test: $(solve("test.txt", 2))")
println("Part 1 solution: $(solve("input.txt", 2))")

Part 1 test: 35
Part 1 solution: 5275


# Part 2

In [3]:
println("Part 2 test: $(@time solve("test.txt", 50))")
println("Part 2 solution: $(@time solve("input.txt", 50))")

  0.061839 seconds (1.20 M allocations: 71.678 MiB, 11.23% gc time)
Part 2 test: 3351


  0.288469 seconds (7.10 M allocations: 424.868 MiB, 8.72% gc time)
Part 2 solution: 16482
